In [3]:
# Add this cell before importing mapboxgl
import sys
from IPython.display import display
sys.modules['IPython.core.display'].display = display

from mapboxgl.viz import ChoroplethViz
from mapboxgl.utils import create_color_stops
import requests

In [6]:



BASE_URL = "http://localhost:8000"
MAPBOX_TOKEN = "pk.eyJ1Ijoic21pdGluIiwiYSI6ImNtZGdleTlhMDBtb3oyanE1eWltMGpwb2cifQ.IRKtLVZamkmvUCMuD64Ybg"
params = {
"day": "Monday",
"period": "AM Peak"
}
response = requests.get(f"{BASE_URL}/aggregates/", params=params)
geojson_data = response.json()



In [ ]:
geojson_data

features = [
{
"type": "Feature",
"geometry": f["geometry"],
"properties": {
"average_speed": f["average_speed"],
"road_name": f["road_name"]
}
} for f in geojson_data
]

In [5]:
viz = ChoroplethViz(
{
"type": "FeatureCollection",
"features": features
},
access_token=MAPBOX_TOKEN,
color_property="average_speed",
color_stops=create_color_stops([10, 20, 30, 40, 50],
colors="Reds"),
center=(-81.6557, 30.3322),
zoom=11,
line_width=1.5,
opacity=0.8,
legend_title="Average Speed (mph)"
)
viz.show()

TypeError: string indices must be integers, not 'str'